In [ ]:
import os
import json
import logging
from datetime import datetime
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'your_broker_address:9092',  # Replace with your broker's address
    'sasl.mechanisms': 'PLAIN',                        # SASL mechanism
    'security.protocol': 'SASL_SSL',                   # Security protocol
    'sasl.username': 'your_username',                   # SASL username
    'sasl.password': 'your_password',                   # SASL password
    'group.id': 'your_consumer_group',                 # Consumer group ID
    'auto.offset.reset': 'earliest'                  # Where to start reading messages
}

# Create a Schema Registry client
schema_registry_config = {
    'url': 'https://your_schema_registry_url',  # Replace with your Schema Registry URL
    'basic.auth.user.info': 'your_username:your_password'  # Basic auth credentials
}

# Fetch the latest Avro schema for the value
try:
    subject_name = 'product_updates-value'
    schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str
except Exception as e:
    logging.error(f"Failed to fetch schema: {e}")
    raise

# Create Avro Deserializer for the value
key_deserializer = StringDeserializer('utf_8')
avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

# Function to handle serialization of datetime objects
def datetime_encoder(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    return obj

# Define a function for the consumer
def run_consumer(consumer_id):
    consumer = DeserializingConsumer({
        'bootstrap.servers': kafka_config['bootstrap.servers'],
        'security.protocol': kafka_config['security.protocol'],
        'sasl.mechanisms': kafka_config['sasl.mechanisms'],
        'sasl.username': kafka_config['sasl.username'],
        'sasl.password': kafka_config['sasl.password'],
        'key.deserializer': key_deserializer,
        'value.deserializer': avro_deserializer,
        'group.id': kafka_config['group.id'],
        'auto.offset.reset': kafka_config['auto.offset.reset'],
        'enable.auto.commit': kafka_config['enable.auto.commit'],
    })

    consumer.subscribe(['product_updates'])

    try:
        while True:
            msg = consumer.poll(1.0)

            if msg is None:
                continue
            if msg.error():
                logging.error(f'Consumer {consumer_id} error: {msg.error()}')
                continue
            
            # Change the category column to lowercase
            try:
                msg.value()['category'] = msg.value()['category'].lower()
                
                # Update the price to half if product belongs to 'category a'
                if msg.value()['category'] == 'categorya':
                    msg.value()['price'] = round(msg.value()['price'] * 0.5, 2)

                logging.info(f'Consumer {consumer_id} - Successfully consumed record with key {msg.key()} and value {msg.value()}')
                
                # Optionally write to a shared file
                json_string = json.dumps(msg.value(), default=datetime_encoder)
                with open(f'shared_consumer_{consumer_id}.json', 'a') as file:
                    file.write(json_string + '\n')

                # Commit the offset manually after processing the message
                consumer.commit(msg)
            
            except Exception as e:
                logging.error(f"Consumer {consumer_id} - Error processing message: {e}")

    except KeyboardInterrupt:
        logging.info(f"Consumer {consumer_id} interrupted by user.")
    finally:
        consumer.close()


In [ ]:
run_consumer(1)

In [ ]:
run_consumer(2)


In [ ]:
run_consumer(3)


In [ ]:
run_consumer(4)
